## Testing @osanders python optimizations



In [1]:
# Example Data
INHERIT = 'inherit'
eg_runtime = {
    'ANIMALS': {},
    'CHORDATA': {INHERIT: 'ANIMALS'},
    'MAMMELIA': {INHERIT: 'CHORDATA'},
    'FARM': {},
    'sheep': {INHERIT: ['FARM', 'MAMMELIA']},
}
 
eg_linearized_ancestors = {
    'ANIMALS': ['ANIMALS', 'root'],
    'CHORDATA': ['CHORDATA', 'ANIMALS', 'root'],
    'MAMMELIA': ['MAMMELIA', 'CHORDATA', 'ANIMALS', 'root'],
    'FARM': ['FARM', 'root'],
    'sheep': ['sheep', 'FARM', 'MAMMELIA', 'CHORDATA', 'ANIMALS', 'root'],
    'root': ['root']
}

eg_descendants = {
    'root': ['ANIMALS', 'CHORDATA', 'MAMMELIA', 'FARM', 'sheep'],
    'ANIMALS': ['CHORDATA', 'MAMMELIA', 'sheep'], 
    'CHORDATA': ['MAMMELIA', 'sheep'], 
    'FARM': ['sheep'],
    'MAMMELIA': ['sheep']
}


In [2]:
def old_flatten(names, lin_ancestors):
    descendents = {}
    for name in names:
        ancestors = lin_ancestors[name]
        for parent in ancestors[1:]:
            if parent not in descendents:
                descendents[parent] = []
            if name not in descendents[parent]:
                descendents[parent].append(name)
    return descendents

In [3]:
assert old_flatten(eg_runtime, eg_linearized_ancestors) == eg_descendants

In [4]:
def new_flatten(names, lin_ancestors):
    descendents = {}
    for name in names:
        ancestors = lin_ancestors[name]
        for parent in ancestors[1:]:
            descendents.setdefault(parent, set()).add(name)
    return descendents    

In [5]:
assert new_flatten(eg_runtime, eg_linearized_ancestors) == \
{k: set(v) for k, v in eg_descendants.items()}

In [6]:
def stretch_graph(len_):
    """create arbitrarily large graphs"""
    runtime = eg_runtime.copy()
    lin_ancestors = eg_linearized_ancestors.copy()
    runtime['0'] = {}
    lin_ancestors['0'] = ['0', 'root']
    this_ancestors = []
    for i in range(len_):
        this_ancestors.append(str(i))
        runtime[str(i+1)] = {INHERIT: str(i)}
        lin_ancestors['sheep'].append(str(i))
        lin_ancestors[str(i+1)] = this_ancestors + ['root', str(i+1)]
    return runtime, lin_ancestors

In [9]:
stretched_graphs = {i: stretch_graph(i) for i in [1, 16, 32, 64, 128, 256, 1024]}

In [10]:
for size, thing in stretched_graphs.items():
    print(size)
    %timeit old_flatten(*thing)
    %timeit new_flatten(*thing)
    

1
279 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
283 µs ± 899 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
16
308 µs ± 3.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
312 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
32
438 µs ± 5.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
367 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
64
1.2 ms ± 44.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
597 µs ± 2.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
128
5.62 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.53 ms ± 4.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
256
38 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.23 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1024
2.28 s ± 9.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
150 ms ± 5.95 ms per loop (mean ± std. dev.